In [1]:
# Jupyter notebook helper: import another .ipynb as module
import sys, os, pathlib, types

def import_ipynb(ipynb_path: str):
    """
    다른 .ipynb 파일을 모듈처럼 불러오기 위한 헬퍼.
    예: utils = import_ipynb("RF_LGBM.ipynb")
    """
    import nbformat
    from IPython.core.interactiveshell import InteractiveShell

    ipynb_path = os.path.abspath(ipynb_path)
    if not os.path.exists(ipynb_path):
        raise FileNotFoundError(f"ipynb module not found: {ipynb_path}")

    nb = nbformat.read(ipynb_path, as_version=4)
    shell = InteractiveShell.instance()
    module_name = pathlib.Path(ipynb_path).stem
    mod = types.ModuleType(module_name)
    mod.__file__ = ipynb_path
    sys.modules[module_name] = mod

    for cell in nb.cells:
        if cell.cell_type == "code":
            code = shell.input_transformer_manager.transform_cell(cell.source)
            exec(code, mod.__dict__)

    return mod

# 워킹 디렉토리를 이 노트북 위치로 맞춰두는 게 안전함
os.chdir(os.path.dirname(os.path.abspath("__file__")))


In [2]:
import os, sys, warnings
from typing import Tuple
from pathlib import Path

import numpy as np
import pandas as pd

# plotting
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    average_precision_score, precision_recall_curve, auc,
    roc_curve, roc_auc_score,
    precision_score, recall_score, f1_score
)

import joblib


In [3]:
# ------------------------
# Default Config
# ------------------------
CFG = dict(
    data_path="dataset/train_features_60m.csv",
    probs_path_candidates=["dataset/holdout_probs.csv", "holdout_probs.csv"],
    ts_col="datetime",
    label_col="passorfail",
    id_cols=["_grp"],
    # window/label settings (mirror the user's run)
    seq_len=300, stride=1,
    label_mode="pre_event",
    label_delta_minutes=180,
    pos_dilate_back_steps=6,
    resample="",
    # split
    val_holdout_days=1.5,
    embargo_minutes=30,
    min_pos_train=10,
    min_pos_holdout=300,
    target_train_frac=0.7,
    train_frac_weight=10.0,
    # policy / thresholds
    post_k=4,
    open_th=0.57,
    close_th=0.33,
    cooldown_min=120,
    # soft-vote weights
    w_tab=0.6,
    w_dl=0.4,
    # cascade gate
    gate_th=0.5,
    # selection constraint
    precision_floor=0.90,
    # output
    out_root="evals_ensemble",
)


In [4]:
# ------------------------
# User notebooks import (TCN, RF/LGBM)
# ------------------------

# 네가 말한 이름 그대로 씀
TCN_NOTEBOOK = "TCN.ipynb"
TAB_NOTEBOOK = "RF_LGBM.ipynb"

# TCN 유틸 가져오기
tcn = import_ipynb(TCN_NOTEBOOK)

# 필요하면 탭모델 관련 유틸도 불러둘 수 있음
# tab_mod = import_ipynb(TAB_NOTEBOOK)


[OK] imports loaded.
[OK] labeling & auto-feature funcs ready.
[OK] sequence builder ready.
[OK] split funcs ready.
[OK] event-based split ready.
[OK] dataset & sampler ready.
[OK] model & loss defined.
[OK] postprocess & eval helpers ready.
[OK] train/eval functions ready.
[DEBUG] total samples=17,265 | positives=2,286
[SPLIT] cutoff=2020-10-30 13:08:44 | embargo=30min | train_win=8806 (pos=1599) | holdout_win=7502 (pos=328) | OK (nudged)
[PREP] drop zero-variance cols on train: 18 dropped
[SPLIT-FINAL] train windows=8,806 (pos=1,599) | holdout windows=7,502 (pos=328)
[INFO] Focal params: gamma=2.0, alpha_pos=0.8184, pos_weight=4.507
Epoch 001 | train_loss=0.022143 | AP=0.955787
Epoch 002 | train_loss=0.001164 | AP=0.934540
Epoch 003 | train_loss=0.000512 | AP=0.881917
[EARLY-STOP] patience=2, best_AP=0.955787
[SAVE] holdout probabilities → dataset/holdout_probs.csv
[EVALS] Saved: evals/*.png, metrics.txt
[SAVE] model artifacts (single file) → models/model_artifacts.pkl
[DONE] trainin

In [5]:
def load_base_dataframe(path: str, ts_col: str, label_col: str, id_cols: list) -> Tuple[pd.DataFrame, list]:
    df = pd.read_csv(path)
    if ts_col not in df.columns:
        raise ValueError(f"ts_col '{ts_col}' not found in {path}")
    if label_col not in df.columns:
        raise ValueError(f"label_col '{label_col}' not found in {path}")
    df[ts_col] = pd.to_datetime(df[ts_col])
    df = df.sort_values(ts_col).reset_index(drop=True)
    excl = set([ts_col, label_col] + (id_cols or []))
    feat_cols = [c for c in df.columns if c not in excl and pd.api.types.is_numeric_dtype(df[c])]
    if not feat_cols:
        raise ValueError("No numeric feature columns found for tabular models.")
    return df, feat_cols


def make_labels_and_windows(tcn, df: pd.DataFrame, feat_cols: list, cfg: dict):
    if cfg["resample"]:
        tmp = df.set_index(cfg["ts_col"])
        agg = {c: "mean" for c in feat_cols}; agg[cfg["label_col"]] = "max"
        df = tmp.resample(cfg["resample"]).agg(agg).reset_index()
        excl = set([cfg["ts_col"], cfg["label_col"]] + (cfg["id_cols"] or []))
        feat_cols = [c for c in df.columns if c not in excl and pd.api.types.is_numeric_dtype(df[c])]

    if cfg["label_mode"] != "pre_event":
        raise NotImplementedError("only pre_event supported")
    y = tcn.make_pre_event_labels(
        df, cfg["ts_col"], cfg["label_col"],
        delta_min=cfg["label_delta_minutes"],
        pos_dilate_back_steps=cfg["pos_dilate_back_steps"],
        resample_minutes=1
    )

    seqs_all, X_dummy, ts_min = tcn.build_windows(
        df, cfg["ts_col"], feat_cols, y,
        seq_len=cfg["seq_len"], stride=cfg["stride"],
        pre_event_minutes=cfg["label_delta_minutes"]
    )

    tr_mask_pts, ho_mask_pts = tcn.embargo_posaware_split(
        df, cfg["ts_col"], y,
        holdout_days=cfg["val_holdout_days"], embargo_minutes=cfg["embargo_minutes"],
        min_pos_train=cfg["min_pos_train"], min_pos_holdout=cfg["min_pos_holdout"],
        seqs_for_split=seqs_all, min_train_windows=32, min_holdout_windows=32,
        target_train_frac=cfg["target_train_frac"], train_frac_weight=cfg["train_frac_weight"]
    )
    tr_idx = [i for i, s in enumerate(seqs_all) if tr_mask_pts[s.start] and tr_mask_pts[s.end - 1]]
    ho_idx = [i for i, s in enumerate(seqs_all) if ho_mask_pts[s.start] and ho_mask_pts[s.end - 1]]
    win_end_row = np.array([s.end - 1 for s in seqs_all], dtype=int)
    ho_end_rows = win_end_row[ho_idx]

    return df, feat_cols, y, seqs_all, tr_idx, ho_idx, ho_end_rows


def _select_feature_columns(df: pd.DataFrame, model, cfg):
    if hasattr(model, "feature_names_in_"):
        cols = [c for c in model.feature_names_in_ if c in df.columns]
        if cols: return cols
    if hasattr(model, "booster_") and hasattr(model.booster_, "feature_name"):
        try:
            names = list(model.booster_.feature_name())
            cols = [c for c in names if c in df.columns]
            if cols: return cols
        except Exception:
            pass
    excl = set([cfg["ts_col"], cfg["label_col"]] + (cfg["id_cols"] or []))
    return [c for c in df.columns if c not in excl and pd.api.types.is_numeric_dtype(df[c])]


def predict_proba_tabular(df_hold: pd.DataFrame, cfg: dict):
    rf_path  = "models/model_RF_calibrated.pkl" if Path("models/model_RF_calibrated.pkl").exists() else "model_RF_calibrated.pkl"
    lgb_path = "models/model_LightGBM.pkl"      if Path("models/model_LightGBM.pkl").exists()      else "model_LightGBM.pkl"
    models = []
    if Path(rf_path).exists():
        try: models.append(joblib.load(rf_path))
        except Exception as e: warnings.warn(f"RF load failed: {e}")
    if Path(lgb_path).exists():
        try: models.append(joblib.load(lgb_path))
        except Exception as e: warnings.warn(f"LGBM load failed: {e}")
    if not models:
        raise FileNotFoundError("No tabular models could be loaded.")

    feat_sets = [set(_select_feature_columns(df_hold, m, cfg)) for m in models]
    common_feats = set.intersection(*feat_sets) if feat_sets else set()
    if not common_feats: common_feats = feat_sets[0]
    cols = sorted(list(common_feats))

    X = df_hold[cols].copy()
    imp = SimpleImputer(strategy="median")
    X_imp_arr = imp.fit_transform(X)
    X_imp = pd.DataFrame(X_imp_arr, columns=X.columns, index=X.index)

    probs = []
    for m in models:
        p = m.predict_proba(X_imp)
        if p.ndim == 2 and p.shape[1] == 2:
            p = p[:, 1]
        probs.append(p.astype(float))
    P_tab = np.mean(np.vstack(probs), axis=0) if len(probs) > 1 else probs[0]
    return P_tab, cols


def hysteresis_cooldown(prob: np.ndarray, win_ends_ts: np.ndarray,
                        open_th: float, close_th: float, cooldown_min: int) -> np.ndarray:
    assert prob.shape[0] == len(win_ends_ts)
    y = np.zeros(len(prob), dtype=int)
    state = 0; last_alarm_end_ts = None
    for i, p in enumerate(prob):
        t = win_ends_ts[i]
        if state == 0:
            ok_cool = True
            if last_alarm_end_ts is not None:
                ok_cool = (t >= last_alarm_end_ts + np.timedelta64(int(cooldown_min), 'm'))
            if ok_cool and (p >= open_th):
                state = 1; y[i] = 1
        else:
            y[i] = 1
            if p <= close_th:
                state = 0
                last_alarm_end_ts = t
    return y


def k_consecutive_filter(yhat_bin: np.ndarray, k: int) -> np.ndarray:
    if k <= 1: return yhat_bin
    y = yhat_bin.copy(); n = len(y); i = 0
    while i < n:
        if y[i] == 1:
            j = i
            while j < n and y[j] == 1: j += 1
            if (j - i) < k: y[i:j] = 0
            i = j
        else: i += 1
    return y


def save_eval_plots(y_true, p_tab, p_dl, p_ens, outdir: Path, tag: str):
    outdir.mkdir(parents=True, exist_ok=True)
    # PR
    for name, p in [("tab", p_tab), ("dl", p_dl), (tag, p_ens)]:
        prec, rec, _ = precision_recall_curve(y_true, p)
        au = auc(rec, prec) if len(rec) > 1 else float("nan")
        plt.figure()
        plt.step(rec, prec, where="post")
        base = (np.asarray(y_true)==1).mean()
        plt.hlines(base, 0, 1, linestyles="dashed")
        plt.xlabel("Recall"); plt.ylabel("Precision")
        plt.title(f"PR Curve [{name}] AUC={au:.4f} (baseline={base:.3f})")
        plt.grid(True, alpha=0.3); plt.tight_layout()
        plt.savefig(outdir / f"pr_{name}.png", dpi=150); plt.close()
    # ROC
    for name, p in [("tab", p_tab), ("dl", p_dl), (tag, p_ens)]:
        if (np.max(y_true) != np.min(y_true)):
            fpr, tpr, _ = roc_curve(y_true, p)
            aucv = roc_auc_score(y_true, p)
            plt.figure()
            plt.plot(fpr, tpr, label=f"AUC={aucv:.4f}")
            plt.plot([0,1],[0,1],"--")
            plt.xlabel("FPR"); plt.ylabel("TPR")
            plt.title(f"ROC Curve [{name}]")
            plt.legend(); plt.grid(True, alpha=0.3); plt.tight_layout()
            plt.savefig(outdir / f"roc_{name}.png", dpi=150); plt.close()
    # Hist
    for name, p in [("tab", p_tab), ("dl", p_dl), (tag, p_ens)]:
        plt.figure()
        y = np.asarray(y_true).astype(int)
        if (y==1).any(): plt.hist(p[y==1], bins=30, alpha=0.6, label="pos")
        if (y==0).any(): plt.hist(p[y==0], bins=30, alpha=0.6, label="neg")
        plt.xlabel("Predicted probability"); plt.ylabel("Count")
        plt.title(f"Probability Histogram [{name}]"); plt.legend(); plt.tight_layout()
        plt.savefig(outdir / f"hist_{name}.png", dpi=150); plt.close()


In [6]:
from pathlib import Path
import numpy as np
import pandas as pd
import warnings

def run_pipeline(mode: str,
                 w_tab: float = None,
                 w_dl: float = None,
                 gate_th: float = None,
                 precision_floor: float = 0.90,
                 out_root: str = "evals_ensemble"):
    cfg = CFG.copy()
    cfg["precision_floor"] = precision_floor
    cfg["out_root"] = out_root

    if mode == "soft":
        assert w_tab is not None and w_dl is not None
        cfg["w_tab"] = w_tab
        cfg["w_dl"] = w_dl
    elif mode == "cascade":
        assert gate_th is not None
        cfg["gate_th"] = gate_th
    else:
        raise ValueError(f"unknown mode: {mode}")

    outdir = Path(out_root) / mode
    outdir.mkdir(parents=True, exist_ok=True)

    df, feat_cols = load_base_dataframe(
        cfg["data_path"], cfg["ts_col"], cfg["label_col"], cfg["id_cols"]
    )

    df2, feat_cols2, y, seqs_all, tr_idx, ho_idx, ho_end_rows = make_labels_and_windows(
        tcn, df.copy(), feat_cols, dict(
            ts_col=cfg["ts_col"], label_col=cfg["label_col"], id_cols=cfg["id_cols"],
            resample=cfg["resample"],
            label_mode=cfg["label_mode"],
            label_delta_minutes=cfg["label_delta_minutes"],
            pos_dilate_back_steps=cfg["pos_dilate_back_steps"],
            seq_len=cfg["seq_len"], stride=cfg["stride"],
            val_holdout_days=cfg["val_holdout_days"],
            embargo_minutes=cfg["embargo_minutes"],
            min_pos_train=cfg["min_pos_train"], min_pos_holdout=cfg["min_pos_holdout"],
            target_train_frac=cfg["target_train_frac"], train_frac_weight=cfg["train_frac_weight"],
        )
    )

    ts_vals = df2[cfg["ts_col"]].values.astype("datetime64[ns]")
    ho_end_ts = ts_vals[ho_end_rows]

    probs_path = None
    for cand in cfg["probs_path_candidates"]:
        if Path(cand).exists():
            probs_path = cand
            break
    if probs_path is None:
        raise FileNotFoundError("holdout_probs.csv not found in candidates: " + ", ".join(cfg["probs_path_candidates"]))
    tcn_df = pd.read_csv(probs_path)
    p_dl = tcn_df["p_hat"].to_numpy() if "p_hat" in tcn_df.columns else tcn_df.iloc[:, -1].to_numpy()

    y_hold = np.array([seqs_all[i].label for i in ho_idx], dtype=int)

    n = min(len(p_dl), len(y_hold))
    if len(p_dl) != len(y_hold):
        warnings.warn(f"[{mode}] Length mismatch: {len(p_dl)} vs {len(y_hold)} → trunc {n}")
    p_dl = p_dl[:n]
    y_hold = y_hold[:n]
    ho_end_ts = ho_end_ts[:n]
    hold_rows = ho_end_rows[:n]
    df_hold = df2.iloc[hold_rows].copy()

    p_tab, used_cols = predict_proba_tabular(df_hold, cfg)
    p_tab = p_tab[:n]

    if mode == "cascade":
        mask = (p_dl >= cfg["gate_th"]).astype(float)
        p_ens = mask * p_tab
        tag = "cascade"
        print(f"[{tag.upper()}] gate_th={cfg['gate_th']:.3f} | suspicious={int(mask.sum())}/{len(mask)}")
    else:
        p_ens = (cfg["w_tab"] * p_tab + cfg["w_dl"] * p_dl).astype(float)
        tag = "soft"

    yb_hys = hysteresis_cooldown(
        p_ens, ho_end_ts,
        cfg["open_th"], cfg["close_th"], cfg["cooldown_min"]
    )
    yb = k_consecutive_filter(yb_hys, cfg["post_k"])

    from sklearn.metrics import average_precision_score, precision_recall_curve, precision_score, recall_score, f1_score

    ap_tab = average_precision_score(y_hold, p_tab)
    ap_dl  = average_precision_score(y_hold, p_dl)
    ap_ens = average_precision_score(y_hold, p_ens)

    prec, rec, thr = precision_recall_curve(y_hold, p_ens)
    thr = np.append(thr, 1.0)
    best_f1, best_t = -1.0, None
    for t in thr:
        yy = (p_ens >= t).astype(int)
        yy = k_consecutive_filter(yy, cfg["post_k"])
        P = precision_score(y_hold, yy, zero_division=0)
        R = recall_score(y_hold, yy, zero_division=0)
        if cfg["precision_floor"] is not None and P < cfg["precision_floor"]:
            continue
        f1v = (2 * P * R / (P + R)) if (P + R) > 0 else 0.0
        if f1v > best_f1:
            best_f1, best_t = f1v, t

    save_eval_plots(y_hold, p_tab, p_dl, p_ens, outdir, tag)

    lines = []
    lines.append(f"Mode: {mode}")
    lines.append(f"AP_tab: {ap_tab:.6f}")
    lines.append(f"AP_dl: {ap_dl:.6f}")
    lines.append(f"AP_{tag}: {ap_ens:.6f}")
    if best_t is not None:
        yy = (p_ens >= best_t).astype(int)
        yy = k_consecutive_filter(yy, cfg["post_k"])
        P = precision_score(y_hold, yy, zero_division=0)
        R = recall_score(y_hold, yy, zero_division=0)
        F = f1_score(y_hold, yy, zero_division=0)
        lines.append(f"Best_threshold_under_precision_floor({cfg['precision_floor']}): {best_t:.6f}")
        lines.append(f"Precision@best: {P:.6f}")
        lines.append(f"Recall@best: {R:.6f}")
        lines.append(f"F1@best: {F:.6f}")
    else:
        lines.append("No threshold met precision_floor")

    with open(outdir / f"metrics_{tag}.txt", "w", encoding="utf-8") as f:
        f.write("\n".join(lines))

    dbg = pd.DataFrame({
        "y_true": y_hold,
        "p_tab": p_tab,
        "p_dl": p_dl,
        "p_ens": p_ens,
        cfg["ts_col"]: ho_end_ts.astype("datetime64[ns]")
    })
    dbg.to_csv(outdir / f"holdout_probs_{tag}.csv", index=False)

    print(f"[{tag.upper()}] Saved plots & metrics to {outdir}")


# 실제 실행: 원래 CLI에서 두 번 때리던 거 그대로 노트북에서 한 번에
run_pipeline(
    mode="cascade",
    gate_th=0.55,
    precision_floor=0.90,
    out_root="evals_ensemble",
)

run_pipeline(
    mode="soft",
    w_tab=0.7,
    w_dl=0.3,
    precision_floor=0.90,
    out_root="evals_ensemble",
)


[SPLIT] cutoff=2020-10-30 13:08:44 | embargo=30min | train_win=8806 (pos=1599) | holdout_win=7502 (pos=328) | OK (nudged)


/opt/conda/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=2.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.0
[LightGBM] [Warning] lambda_l1 is set=1.0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.0
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[CASCADE] gate_th=0.550 | suspicious=287/7502
[CASCADE] Saved plots & metrics to evals_ensemble/cascade
[SPLIT] cutoff=2020-10-30 13:08:44 | embargo=30min | train_win=8806 (pos=1599) | holdout_win=7502 (pos=328) | OK (nudged)


/opt/conda/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=2.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.0
[LightGBM] [Warning] lambda_l1 is set=1.0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.0
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[SOFT] Saved plots & metrics to evals_ensemble/soft
